In [193]:
import numpy as np

In [194]:
# Helper functions

# def softmax(x):
#     """Compute the softmax function for each row of the input x.

#     Arguments:
#     x -- A D dimensional vector or N x D dimensional numpy matrix.

#     Return:
#     x -- after application of the softmax function
#     """
#     orig_shape = x.shape

#     if len(x.shape) > 1:
#         # x is a matrix
#         col_max = np.max(x, axis=1)                # find max across columns
#         x -= np.reshape(col_max, (x.shape[0],1))   # need to reshape matching the initial row size
#         num = np.exp(x)
#         denum = np.reshape(np.sum(np.exp(x), axis=1), (x.shape[0], 1)) # need to reshape before division
#         x = np.divide(num, denum)
#     else:
#         # x is a vector
#         x -= np.max(x)
#         x = np.divide(np.exp(x), np.sum(np.exp(x)))

#     assert x.shape == orig_shape
#     return x

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [195]:
# Notes for RNN cell step

In [196]:
def rnn_step_forward(parameters, a_prev, x):
    
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']
    a_next = np.tanh(np.dot(Wax, x) + np.dot(Waa, a_prev) + b) 
    p_t = softmax(np.dot(Wya, a_next) + by) 
    
    return a_next, p_t

def rnn_step_backward(dy, gradients, parameters, x, a, a_prev):
    
    gradients['dWya'] += np.dot(dy, a.T)
    gradients['dby'] += dy
    da = np.dot(parameters['Wya'].T, dy) + gradients['da_next'] # backprop into h
    daraw = (1 - a * a) * da # backprop through tanh nonlinearity
    gradients['db'] += daraw
    gradients['dWax'] += np.dot(daraw, x.T)
    gradients['dWaa'] += np.dot(daraw, a_prev.T)
    gradients['da_next'] = np.dot(parameters['Waa'].T, daraw)
    return gradients

def update_parameters(parameters, gradients, lr):

    parameters['Wax'] += -lr * gradients['dWax']
    parameters['Waa'] += -lr * gradients['dWaa']
    parameters['Wya'] += -lr * gradients['dWya']
    parameters['b']  += -lr * gradients['db']
    parameters['by']  += -lr * gradients['dby']
    return parameters

def rnn_forward(X, Y, a0, parameters, vocab_size = 27):
    
    # Initialize x, a and y_hat as empty dictionaries
    x, a, y_hat = {}, {}, {}
    
    a[-1] = np.copy(a0)
    
    loss = 0
    
    for t in range(len(X)):
        
        # Set x[t] to be the one-hot vector representation of the t'th character in X.
        # if X[t] == None, we just have x[t]=0. This is used to set the input for the first timestep to the zero vector. 
        x[t] = np.zeros((vocab_size,1)) 
        if (X[t] != None):
            x[t][X[t]] = 1
        
        # Run one step forward of the RNN
        a[t], y_hat[t] = rnn_step_forward(parameters, a[t-1], x[t])
        
        # Update the loss by substracting the cross-entropy term of this time-step from it.
        loss -= np.log(y_hat[t][Y[t],0])
        
    cache = (y_hat, a, x)
        
    return loss, cache

def rnn_backward(X, Y, parameters, cache):
    # Initialize gradients as an empty dictionary
    gradients = {}
    
    # Retrieve from cache and parameters
    (y_hat, a, x) = cache
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']
    
    # each one should be initialized to zeros of the same dimension as its corresponding parameter
    gradients['dWax'], gradients['dWaa'], gradients['dWya'] = np.zeros_like(Wax), np.zeros_like(Waa), np.zeros_like(Wya)
    gradients['db'], gradients['dby'] = np.zeros_like(b), np.zeros_like(by)
    gradients['da_next'] = np.zeros_like(a[0])
    
    # Backpropagate through time
    for t in reversed(range(len(X))):
        dy = np.copy(y_hat[t])
        dy[Y[t]] -= 1
        gradients = rnn_step_backward(dy, gradients, parameters, x[t], a[t], a[t-1])
    
    return gradients, a

In [197]:
def clip(gradients, maxValue):
    
    dWaa, dWax, dWya, db, dby = gradients['dWaa'], gradients['dWax'], gradients['dWya'], gradients['db'], gradients['dby']
   
    # clip to mitigate exploding gradients, loop over [dWax, dWaa, dWya, db, dby].
    for gradient in [dWax, dWaa, dWya, db, dby]:
        gradient.clip(-maxValue, maxValue, out=gradient)
        
    gradients = {"dWaa": dWaa, "dWax": dWax, "dWya": dWya, "db": db, "dby": dby}
    
    return gradients

In [198]:
def sample(parameters, char_to_ix):
    """
    Sample a sequence of characters according to a sequence of probability distributions output of the RNN
    """
    
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']
    vocab_size = by.shape[0]
    n_a = Waa.shape[1]
    
    # Create the one-hot vector x for the first character (initializing the sequence generation).
    x = np.zeros((vocab_size,1))
    print(vocab_size)
#     x = np.eye(vocab_size)[0]

    a_prev = np.zeros((n_a,1))
    
    # Create an empty list of indices, contains the list of indices of the characters to generate
    indices = []
    
    # idx is a flag to detect a newline character
    idx = -1 
    
    # loop over time-steps t. At each time-step, sample a character from a probability distribution and append 
    # its index to "indices" 
    counter = 0
    newline_character = char_to_ix['\n']
    
    while (idx != newline_character and counter != 50):
        
        # Step 2: Forward propagate x using the equations (1), (2) and (3)
#         print(Wax.shape, x.shape)
#         print(Waa.shape, a_prev.shape)
        a = np.tanh(np.dot(Wax, x) + np.dot(Waa, a_prev) + b)
        z = np.dot(Wya, a) + by
        y = softmax(z)
        
        
        # sample the index of a character within the vocabulary from the probability distribution y
        idx = np.random.choice(range(vocab_size), p=y.ravel())

        # sppend the index to "indices"
        indices.append(idx)
        
        # overwrite the input character as the one corresponding to the sampled index.
        x = np.zeros((vocab_size,1))
        x[idx] = 1
        
        a_prev = a
        
        counter +=1
        
    if (counter == 50):
        indices.append(char_to_ix['\n'])
    
    return indices

In [199]:
def optimize(X, Y, a_prev, parameters, learning_rate = 0.01):
    """
    Execute one step of the optimization to train the model.
    """
    
    
    # Forward propagate through time
    loss, cache = rnn_forward(X, Y, a_prev, parameters)
    
    # Backpropagate through time
    gradients, a = rnn_backward(X, Y, parameters, cache)
    
    # Clip your gradients between -5 (min) and 5 (max)
    gradients = clip(gradients, 5)
    
    # Update parameters
    parameters = update_parameters(parameters, gradients, learning_rate)
        
    return loss, gradients, a[len(X)-1]

In [200]:
def model(data, ix_to_char, char_to_ix, num_iterations = 35000, n_a = 50, dino_names = 7, vocab_size = 27):
    """
    Trains the model and generates new words. 
    """
    
    n_x, n_y = vocab_size, vocab_size
    
    parameters = initialize_parameters(n_a, n_x, n_y)
    
    # Build list of all dinosaur names (training examples).
    with open("input/dinos.txt") as f:
        examples = f.readlines()
    examples = [x.lower().strip() for x in examples]
        
    # Initialize the hidden state of your LSTM
    a_prev = np.zeros((n_a, 1))
    
    # Optimization loop
    for j in range(num_iterations):
        index = j % len(examples)
        X = [None] + [char_to_ix[ch] for ch in examples[index]] 
        Y = X[1:] + [char_to_ix["\n"]]
        
        # Perform one optimization step: Forward-prop -> Backward-prop -> Clip -> Update parameters
        # Choose a learning rate of 0.01
        curr_loss, gradients, a_prev = optimize(X, Y, a_prev, parameters, learning_rate = 0.01)
                
        loss = curr_loss

        # Every 2000 Iteration, generate "n" characters thanks to sample() to check if the model is learning properly
        if j % 2000 == 0:
            
            print('Iteration: %d, Loss: %f' % (j, loss) + '\n')
            
            # The number of dinosaur names to print
            for name in range(dino_names):
                sampled_indices = sample(parameters, char_to_ix)
                print_sample(sampled_indices, ix_to_char)
                      
            print('\n')
        
    return parameters

def get_initial_loss(vocab_size, seq_length):
    return -np.log(1.0/vocab_size)*seq_length

In [201]:
def initialize_parameters(n_a, n_x, n_y):
    """
    Initialize parameters with small random values
    """

    Wax = np.random.randn(n_a, n_x)*0.01 # input to hidden
    Waa = np.random.randn(n_a, n_a)*0.01 # hidden to hidden
    Wya = np.random.randn(n_y, n_a)*0.01 # hidden to output
    b = np.zeros((n_a, 1)) # hidden bias
    by = np.zeros((n_y, 1)) # output bias
    
    parameters = {"Wax": Wax, "Waa": Waa, "Wya": Wya, "b": b,"by": by}
    
    return parameters

In [202]:
data = open('input/dinos.txt', 'r').read()
data= data.lower()
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('There are %d total characters and %d unique characters in your data.' % (data_size, vocab_size))

def print_sample(sample_ix, ix_to_char):
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    txt = txt[0].upper() + txt[1:] 
    print ('%s' % (txt, ), end='')

There are 19909 total characters and 27 unique characters in your data.


In [203]:
char_to_ix = { ch:i for i,ch in enumerate(sorted(chars)) }
ix_to_char = { i:ch for i,ch in enumerate(sorted(chars)) }
print(ix_to_char)
print(char_to_ix)

{0: '\n', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'}
{'\n': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26}


In [204]:
parameters = model(data, ix_to_char, char_to_ix)

Iteration: 0, Loss: 46.137999

27
B
27
Rudkwsyheuzvazuugnyezbskenlsoctzdltkaxvk
27
Hyxrmjeauaveiz
27
Nmn
27
Eqosktsbtkdecnwropsiwmjbaxpnqwajwyexpuohnivmvisfxw
27
Onllwhgzcsxpcenvnc
27
Ifbfcudkasnozpobstnulgntm


Iteration: 2000, Loss: 28.344514

27
Tennsavanrys
27
Vopplosaurus
27
Meraderis
27
Aurisaudus
27
Eamdadaurus
27
Reanisaurus
27
Eerosaurfaurus


Iteration: 4000, Loss: 21.931832

27
Onmetontos
27
Simanroph
27
Lonthanannadeepucropiisanrus
27
Moveodos
27
Maenonosaurus
27
Mostos
27
Bedophanlor


Iteration: 6000, Loss: 30.082031

27
Gmctostcutoostinotlonttopatrus
27
Wimithonya
27
Tlmonnostutfnitan
27
Suodatos
27
Saviotownagorinter
27
Sarenda
27
S


Iteration: 8000, Loss: 12.627768

27
Rami
27
Urus
27
Bekeiatoos
27
Lemarallachusurasaurus
27
Auiisaurus
27
Lophetoo
27
Anosatrus


Iteration: 10000, Loss: 20.212594

27
Nanosaurus
27
Konisaurus
27
Iatirvanons
27
Alesaurus
27
Galiosaurus
27
Eucong
27
Laimongos


Iteration: 12000, Loss: 33.090540

27
Pimuosaurus
27
Hoiusiliakis
27
Sugora
27
